# GGG基盤上の薄膜

In [1]:
#------------------------
# Google Colab上でのみ実行
#------------------------
import time
%env TOKEN=*************************************
! git clone https://$$TOKEN@github.com/konnitiha3/MOD2NN.git

import sys
sys.path.append('/content/MOD2NN')

from google.colab import drive
drive.mount('/content/drive')

!pip install GPyOpt

env: TOKEN=*************************************
Cloning into 'MOD2NN'...
remote: Enumerating objects: 1025, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 1025 (delta 81), reused 151 (delta 57), pack-reused 829
Receiving objects: 100% (1025/1025), 36.27 MiB | 27.09 MiB/s, done.
Resolving deltas: 100% (560/560), done.
Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 3.5 MB/s 
     |████████████████████████████████| 959 kB 15.2 MB/s 
     |████████████████████████████████| 71 kB 10.4 MB/s 
  Created wheel for GPyOpt: filename=GPyOpt-1.2.6-py3-none-any.whl size=83609 sha256=f06ec1308699ebf744cb142829bcd04d2ed10fc656533325a33443a4c36b7519
  Stored in directory: /root/.cache/pip/wheels/e6/fa/d1/f9652b5af79f769a0ab74dbead7c7aea9a93c6bc74543fd3ec
  Created wheel for GPy: filename=GPy-1.10.0-cp37-cp37m

In [2]:
import time
import os
import json
import sys
import re
import glob
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import GPyOpt

from Faraday.two_dim.module.lib.layers import *
from Faraday.two_dim.module.lib import regularizer

tf.random.set_seed(1)

print("TensorFlow:", tf.__version__)
print("Python:", sys.version)

plt.rcParams['font.size'] = 18

TensorFlow: 2.8.2
Python: 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]


In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-5000:]
y_val = y_train[-5000:]
x_train = x_train[:-5000]
y_train = y_train[:-5000]

11501568/11490434 [==============================] - 0s 0us/step


## MNISTDetector標準モデル
ShiftL1:L1 = $1.0^{-5}$

In [4]:
#@title デフォルトのタイトル テキスト
wavelength = 532.0e-9 #@param {type:"number"}
d = 1.0e-6 #@param {type:"number"}
n = 1.5 #@param {type:"number"}

In [5]:
def create_model(**kwargs):
    shape = (100, 100)
    inputs = tf.keras.Input((28, 28))
    theta = -2.105 * np.pi / 180
    eta = np.arctan(0.9785 * np.pi/180)/2
    l1=1.0e-5
    x = ImageResizing(shape)(inputs)
    x = ImageBinarization(0.5, 0.0, 1.0)(x)
    x = IntensityToElectricField(shape)(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta, kernel_regularizer=regularizer.ShiftL1Regularizer(l1, np.pi/2))(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.5e-3, d=d, n=2.0, method='expand')(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta, kernel_regularizer=regularizer.ShiftL1Regularizer(l1, np.pi/2))(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.5e-3, d=d, n=1.00, method='expand')(x)
    # x = Polarizer(shape)(x)
    #x =ElectricFieldToIntensity(shape)(x)
    #x = MNISTFilter(shape)(x)
    x = FaradayRotationByStokes(shape)(x)
    # x = Argument(shape)(x)
    x = MNISTDetector(10)(x)
    x = tf.keras.layers.Softmax()(x)
    model = tf.keras.Model(inputs, x)
    return model

In [6]:
model = create_model()
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,  # category: sparse_categorical_crossentropy
              metrics=['accuracy'])

epochs = 50
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    min_delta=0.05,
    patience=2,
)

model_name = "20220809_1"
cholab_path = "/content/drive/MyDrive/D2NN/"
checkpoint_path = cholab_path + "checkpoint/" + model_name + "/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# チェックポイントコールバックを作る
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

logdir = os.path.join(cholab_path +"logs", model_name)
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

result = model.fit(x_train,
                   y_train,
                   batch_size=64,
                   epochs=epochs,
                   validation_data=(x_val, y_val),
                   callbacks=[cp_callback, tensorboard_callback]
                   )

path = cholab_path + "trained_model/"+ model_name
model.save(path)

df = pd.DataFrame(result.history)
df.to_csv(path + "/history.csv")

with open(path + "/config.json", 'w') as f:
    json.dump(model.get_config(), f, indent=4)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 image_resizing (ImageResizi  (None, 100, 100)         0         
 ng)                                                             
                                                                 
 image_binarization (ImageBi  (None, 100, 100)         0         
 narization)                                                     
                                                                 
 intensity_to_electric_field  (None, 2, 100, 100)      0         
  (IntensityToElectricField)                                     
                                                                 
 mo (MO)                     (None, 2, 100, 100)       10000     
                                                             

## 検出距離最適化最適化 (BO)
2値化後のloss最小化最小化


In [7]:
def create_model(**kwargs):
    shape = (100, 100)
    inputs = tf.keras.Input((28, 28))
    theta = -2.105 * np.pi / 180
    eta = np.arctan(0.9785 * np.pi/180)/2
    l1=1.0e-5
    z = kwargs["z"]
    x = ImageResizing(shape)(inputs)
    x = ImageBinarization(0.5, 0.0, 1.0)(x)
    x = IntensityToElectricField(shape)(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta, kernel_regularizer=regularizer.ShiftL1Regularizer(l1, np.pi/2))(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.5e-3, d=d, n=2.0, method='expand')(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta, kernel_regularizer=regularizer.ShiftL1Regularizer(l1, np.pi/2))(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=1.00, method='expand')(x)
    # x = Polarizer(shape)(x)
    #x =ElectricFieldToIntensity(shape)(x)
    #x = MNISTFilter(shape)(x)
    x = FaradayRotationByStokes(shape)(x)
    # x = Argument(shape)(x)
    x = MNISTDetector(10)(x)
    x = tf.keras.layers.Softmax()(x)
    model = tf.keras.Model(inputs, x)
    return model

In [9]:
history = []
count = 0
def train(**kwargs):
    global count
    count += 1
    model = create_model(**kwargs)
    model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.sparse_categorical_crossentropy,  # category: sparse_categorical_crossentropy
                  metrics=['accuracy'])

    epochs = 50
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='accuracy',
        min_delta=0.05,
        patience=2,
    )

    model_name = "20220809_BO/" + str(count)
    cholab_path = "/content/drive/MyDrive/D2NN/"
    checkpoint_path = cholab_path + "checkpoint/" + model_name + "/cp-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)

    # チェックポイントコールバックを作る
    cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=1)

    logdir = os.path.join(cholab_path +"logs", model_name)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    result = model.fit(x_train,
                       y_train,
                       batch_size=64,
                       epochs=epochs,
                       validation_data=(x_val, y_val),
                       callbacks=[cp_callback, tensorboard_callback]
                       )

    path = cholab_path + "trained_model/"+ model_name
    model.save(path)

    df = pd.DataFrame(result.history)
    df.to_csv(path + "/history.csv")

    with open(path + "/config.json", 'w') as f:
        json.dump(model.get_config(), f, indent=4)

    bi_model = tf.keras.models.load_model(path)
    pattern = r'mo'
    mo_layers = []
    for layer in bi_model.layers:
        result = re.match(pattern, layer.name)
        if result:
            mo_layers.append(layer)

    for layer in mo_layers:
        w = layer.get_weights()
        bi_w = np.where(w[0]>0, np.pi/2, -np.pi/2)
        w[0] = bi_w
        layer.set_weights(w)
    evaluate = bi_model.evaluate(x_test, y_test)
    history.append([count, *kwargs.values(), evaluate[0], evaluate[1]])
    df = pd.DataFrame(history)
    df.to_csv("/content/drive/MyDrive/D2NN/trained_model/20220809_BO/bo_history.csv")

    return evaluate

In [ ]:
bounds = [{'name': 'z',  'type': 'continuous',  'domain': (0.1e-3, 5.0e-3)}]
# ベイズ最適化する関数（上記で書いたブラックボックス）を定義します。
# xが入力で、出力はreturnされます。
def f(x):
    print(x)
    evaluation = train(
        z = float(x[:,0]))
    print("loss:{0} \t\t accuracy:{1}".format(evaluation[0], evaluation[1]))
    print(evaluation)
    return evaluation[0]

# 事前探索を行います。
opt_mnist = GPyOpt.methods.BayesianOptimization(f=f, domain=bounds)

# 最適なパラメータを探索します。
opt_mnist.run_optimization(max_iter=50)
print("optimized parameters: {0}".format(opt_mnist.x_opt))
print("optimized loss: {0}".format(opt_mnist.fx_opt))

[[0.00267388]]
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 image_resizing_1 (ImageResi  (None, 100, 100)         0         
 zing)                                                           
                                                                 
 image_binarization_1 (Image  (None, 100, 100)         0         
 Binarization)                                                   
                                                                 
 intensity_to_electric_field  (None, 2, 100, 100)      0         
 _1 (IntensityToElectricFiel                                     
 d)                                                              
                                                                 
 mo_2 (MO)                   (None, 2, 100, 